# Ceneo Scraper

## Struktura opinii w serwisie Ceneo.pl

|Składowa|Selektor|Zmienna|
|--------|--------|--------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation > em|recommendation|
|gwiazdki|span.user-post__score-count|rating|
|treść|div.user-post__text|content|
|lista zalet|div.review-feature__title--positives ~ div.review-feature__item|pros|
|lista wad|div.review-feature__title--negatives ~ div.review-feature__item|cons|
|dla ilu przydatna|button.vote-yes > span|useful|
|dla ilu nieprzydatna|button.vote-no > span|useless|
|data wystawienia|("span.user-post__published > time:nth-child(1)").get("datetime")|publish_date|
|data zakupu|("span.user-post__published > time:nth-child(2)").get("datetime")|purchase_date|

## Biblioteki

In [17]:
import os
import json
import requests
from bs4 import BeautifulSoup

## Funkcja do

In [31]:
def extract_data(ancestor, selector=None, attribute=None, return_list=False):
    if selector:
        if return_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

## Słownik selektorów

In [32]:
selectors = {
    "opinion_id" : (None, "data-entry-id"),
    "author" : ("span.user-post__author-name",),
    "recommendation" : ("span.user-post__author-recomendation > em",),
    "rating" : ("span.user-post__score-count",),
    "content" : ("div.user-post__text",),
    "pros" : ("div.review-feature__title--positives ~ div.review-feature__item", None, True),
    "cons" : ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
    "useful" : ("button.vote-yes > span",),
    "useless" : ("button.vote-no > span",),
    "publish_date" : ("span.user-post__published > time:nth-child(1)", "datetime"),
    "purchase_date" : ("span.user-post__published > time:nth-child(2)", "datetime")
}

## Wysłanie do serwera żądania dostępu do zasobu

In [33]:
#product_id = "156802519"
product_id = input("Podaj kod produktu w serwisie Ceneo.pl")
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

In [34]:
all_opinions = []
while(url):
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key : extract_data(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl/" + extract_data(page_dom, "a.pagination__next", "href", False)
    except (TypeError, AttributeError):
        url = None

In [35]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jsonf:
    json.dump(all_opinions, jsonf, indent=4, ensure_ascii=False)

In [36]:
len(all_opinions)

129